In [1]:
%cd ../data/AIC21_Track3_MTMC_Tracking/
%pwd

/mnt/d/coding/LMMCT/data/AIC21_Track3_MTMC_Tracking


'/mnt/d/coding/LMMCT/data/AIC21_Track3_MTMC_Tracking'

In [2]:
import json
track5_data_path = '../AIC21_Track5_NL_Retrieval/data/train_tracks_2021.json'
with open(track5_data_path) as f:
    train_set = json.load(f)

In [3]:
len(train_set)

2498

In [4]:
# train_set

In [5]:
# Convert to by frame data (using frame as key)
count = 0
train_set_by_frame = {}
for value in train_set.values():
    len_track = len(value['frames'])
    for bbox_id in range(len_track):
        count+=1
        frame_path = value['frames'][bbox_id]
        frame_path = frame_path.replace('img1', 'original_img')
        bbox_coor = tuple(value['boxes'][bbox_id])
        nl = value['nl']
        if frame_path not in train_set_by_frame.keys():
            train_set_by_frame[frame_path] = {}
        # print(bbox_coor)
        train_set_by_frame[frame_path].update({bbox_coor:nl})

In [6]:
# Number of bboxes
count

202550

In [7]:
# Number of frames
len(train_set_by_frame)

74571

In [8]:
train_set_by_frame['train/S01/c001/original_img/000057.jpg']

{(1450,
  337,
  423,
  173): ['A blue pickup truck keeps straight at an intersection.', 'A blue pickup drives through the intersection.', 'A blue Pickup Truck running down the street.']}

In [9]:
# Sort by frame_path
train_set_by_frame_cp = train_set_by_frame
order_train_set_by_frame = {key:train_set_by_frame_cp[key] for key in sorted(train_set_by_frame_cp.keys())}

In [10]:
len(order_train_set_by_frame)

74571

In [11]:
# order_train_set_by_frame

### Extract car type and car color from nl

In [12]:
# # nls = [[nl for bbox, nl in bboxes.items()] for _,bboxes in order_train_set_by_frame.items()]
# nls =[]
# for _,bboxes in order_train_set_by_frame.items():
#     for bbox, nl in bboxes.items():
#         nls.append(nl)


In [13]:
# # number of bboxes
# len(nls)

In [14]:
from collections import Counter
car_type_lib = ['sedan', 'wagon', 'suv', 'couple', 'van', 'pickup', 'truck']
car_col_lib = ['red', 'blue', 'brown', 'gray', 'orange', 'black', 'purple', 'silver', 'green', 'white', 'yellow']

def nl_preprocessor(text):
    
    # Only load 6 first word
    words = text.split(' ')[:6]
    text = (' ').join(words)

    # Join "pick up" into "pickup"
    if "pick up" in text:
        text = text.replace('pick up', 'pickup')
    
    # Remove Truck if pickup present
    if 'pickup' in text:
        text = text.replace('truck', '')
    
    # Remove Truck if pickup present
    if 'pickup' in text:
        text = text.replace('truck', '')
    
    # Change wagon to suv
    if 'wagon' in text:
        text = text.replace('wagon', 'suv')
    
    # Change couple to sedan
    if 'couple' in text:
        text = text.replace('couple', 'sedan')
    
    return text
        
def _att_extractor(text, car_type_lib, car_col_lib):
    words = text.split(' ')[:6]
    words = [word.lower() for word in words]
    car_type = 'z.unknow'
    car_col = 'z.unknow'
    for word in words:
        for c_type in car_type_lib:
            if c_type in word:
                car_type = c_type
        for c_col in car_col_lib:
            if c_col in word:
                car_col = c_col
    return [car_type, car_col]

def voting(votes):
    #Count the votes for persons and stores in the dictionary
    vote_count=Counter(votes)
    
    # if 3 different value for 3 nl or there >= 2 unknow value
    if (len(vote_count) >= 3) or (vote_count['z.unknow']>=2):
        votes = [vote for vote in votes if vote != 'z.unknow']
        votes = ('.').join(votes)
        return [votes, False]

    #Find the maximum number of votes
    max_votes=max(vote_count.values())

    #Search for people having maximum votes and store in a list
    lst=[i for i in vote_count.keys() if vote_count[i]==max_votes]

    #Sort the list and print lexicographical smallest name
    return [sorted(lst)[0], True]
    

def att_extractor(nl, car_type_lib, car_col_lib):
    car_type_votes = []
    car_col_votes =[]
    for text in nl:
        text = nl_preprocessor(text)
        atts = _att_extractor(text,car_type_lib, car_col_lib)
        car_type_votes.append(atts[0])
        car_col_votes.append(atts[1])
    car_type = voting(car_type_votes)
    car_col = voting(car_col_votes)
    return car_type+ car_col

In [15]:
# nls[2]

In [16]:
# nls_per_bbox = [att_extractor(nl) for nl in nls]
# len(nls_per_bbox)

In [17]:
# #  Print for debug
# for i in range(500,5000):
#     print (nls[i])
#     print (nls_per_bbox[i])

In [18]:
# order_train_set_by_frame

In [19]:
order_train_set_by_frame['train/S01/c001/original_img/000056.jpg'][(1513,
  339,
  405,
  175)]

['A blue pickup truck keeps straight at an intersection.',
 'A blue pickup drives through the intersection.',
 'A blue Pickup Truck running down the street.']

In [20]:
import copy
# del(final_train_set_by_frame)
count = 0
false_count = 0
final_train_set_by_frame = copy.deepcopy(order_train_set_by_frame)
for frame_path,bboxes in order_train_set_by_frame.items():
    for bbox, nl in bboxes.items():
        count+=1
        tmp = ['z.unknow']*2 + att_extractor(nl,car_type_lib, car_col_lib)
        tmp = tmp + nl 
        if False in tmp:
            false_count +=1
        final_train_set_by_frame[frame_path][bbox]=tmp
false_count/ count

0.29041224389039744

In [21]:
# final_train_set_by_frame 

In [22]:
# count = 0
# for frame_path,bboxes in final_train_set.items():
#     for bbox, nl in bboxes.items():
#         count += 1
# count

### Get car Id from track 3

In [23]:
%pwd

'/mnt/d/coding/LMMCT/data/AIC21_Track3_MTMC_Tracking'

In [24]:
# Get gt path
from pathlib import Path

gt_paths = []
# root = Path("../../AIC21_Track3_MTMC_Tracking")
root = Path("./")
srcs = [root / 'train', root / 'validation']

for src in srcs:
    for scenario in  src.iterdir():
        for camera in scenario.iterdir():
            gt_path = camera / 'gt' / 'gt.txt'
            gt_paths.append(gt_path)
gt_paths


[PosixPath('train/S01/c001/gt/gt.txt'),
 PosixPath('train/S01/c002/gt/gt.txt'),
 PosixPath('train/S01/c003/gt/gt.txt'),
 PosixPath('train/S01/c004/gt/gt.txt'),
 PosixPath('train/S01/c005/gt/gt.txt'),
 PosixPath('train/S03/c010/gt/gt.txt'),
 PosixPath('train/S03/c011/gt/gt.txt'),
 PosixPath('train/S03/c012/gt/gt.txt'),
 PosixPath('train/S03/c013/gt/gt.txt'),
 PosixPath('train/S03/c014/gt/gt.txt'),
 PosixPath('train/S03/c015/gt/gt.txt'),
 PosixPath('train/S04/c016/gt/gt.txt'),
 PosixPath('train/S04/c017/gt/gt.txt'),
 PosixPath('train/S04/c018/gt/gt.txt'),
 PosixPath('train/S04/c019/gt/gt.txt'),
 PosixPath('train/S04/c020/gt/gt.txt'),
 PosixPath('train/S04/c021/gt/gt.txt'),
 PosixPath('train/S04/c022/gt/gt.txt'),
 PosixPath('train/S04/c023/gt/gt.txt'),
 PosixPath('train/S04/c024/gt/gt.txt'),
 PosixPath('train/S04/c025/gt/gt.txt'),
 PosixPath('train/S04/c026/gt/gt.txt'),
 PosixPath('train/S04/c027/gt/gt.txt'),
 PosixPath('train/S04/c028/gt/gt.txt'),
 PosixPath('train/S04/c029/gt/gt.txt'),


In [25]:
import numpy as np
def xywh_to_xyxy(xywh):
    """Convert [x1 y1 w h] box format to [x1 y1 x2 y2] format."""
    if isinstance(xywh, (list, tuple)):
        # Single box given as a list of coordinates
        assert len(xywh) == 4
        x1, y1 = xywh[0], xywh[1]
        x2 = x1 + np.maximum(0., xywh[2] - 1.)
        y2 = y1 + np.maximum(0., xywh[3] - 1.)
        return (x1, y1, x2, y2)
    elif isinstance(xywh, np.ndarray):
        # Multiple boxes given as a 2D ndarray
        return np.hstack(
            (xywh[:, 0:2], xywh[:, 0:2] + np.maximum(0, xywh[:, 2:4] - 1))
        )
    else:
        raise TypeError('Argument xywh must be a list, tuple, or numpy array.')

def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    bb1 = xywh_to_xyxy(bb1)
    bb2 = xywh_to_xyxy(bb2)
    
    coors = ['x1', 'y1', 'x2', 'y2']
    bb1 = {key:value for key,value in zip(coors,  bb1)}
    bb2 = {key:value for key,value in zip(coors,  bb2)}
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

bb1 = (1509,347,411,160)
bb2 = (1513, 339, 405, 175)
a = get_iou(bb1, bb2)
a

0.9015578947368421

In [26]:
bb1 = [int(x) for x in '863_157_86_68'.split('_')]
bb2 = [int(x) for x in '846_142_118_105'.split('_')]
a = get_iou(bb1, bb2)
a


0.4680309007232084

In [27]:
def match_bbox(query, bboxes, threhold = 0.8):
    bboxes_ls = list(bboxes)
    # print (query)
    # print (bboxes_ls)
    # print (len(bboxes_ls))
    if len(bboxes_ls) == 0: return query
    ious = [get_iou(query, bbox) for bbox in bboxes_ls]
    max_i = np.argmax(ious)
    if ious[max_i] > threhold:
        return bboxes_ls[max_i]
    else:
        return query
        

In [28]:
#final_train_set_by_frame

In [29]:
final_train_set_by_frame['validation/S05/c034/original_img/003225.jpg']

{(953, 222, 182, 111): ['z.unknow',
  'z.unknow',
  'suv',
  True,
  'green',
  True,
  'Suburu forester wagon making a right turn.',
  'A green wagon turns right at the intersection.',
  'A green wagon turns first turns left at an intersection then turns right at a parking lot.'],
 (711, 231, 175, 134): ['z.unknow',
  'z.unknow',
  'suv',
  True,
  'black',
  True,
  'A black SUV keeps straight on a curvy road.',
  'A black MPV turns right behind a red vehicle.',
  'A black SUV right turns left at the intersection behind another black vehicle.'],
 (1644, 357, 274, 156): ['z.unknow',
  'z.unknow',
  'sedan',
  False,
  'black.blue',
  False,
  'A black sedan turns left followed by the Jeep.',
  'A blue car turns left then goes straight.',
  'It is a maroon car that makes a stop before it turns right and drives in the right lane behind other cars.'],
 (1134, 430, 361, 243): ['z.unknow',
  'z.unknow',
  'suv',
  True,
  'black',
  True,
  'Dark suv going straight.',
  'A black SUV runs d

In [30]:
total_count=0
not_match_counter =0
matching_threhold=0.5
final_train_set_by_frame_cp = copy.deepcopy( final_train_set_by_frame)
# Create dict to store val for all tracks (by VehID)
tracks = {VehId: [] for VehId in range(5000)}

# final_train_set_by_frame_cp = {}
for gt_path in gt_paths:
    dir_tmp = gt_path.parents[1] / 'original_img'
    with open(gt_path) as f: lines = f.readlines()
    for line in lines:
        total_count+=1
        # print (line)
        atts = line.split(',')
        atts = [int(val) for val in atts]
        
        # Extract frame_path
        frame_Id = atts[0]
        # frame_Id = str(int(atts[0]) + 1) #frame start from 1 not 0 (match nl cityflow)
        img_name = f'{int(frame_Id):06d}.jpg'
        frame_path = str(dir_tmp/img_name)
        
        # Extract vehId and bbox
        vehId = atts[1]
        bbox_cor = tuple(atts[2:6])
        
        # Feed vehId into final_train_set_by_frame
        # Create empty dict if no frame or bbox_cor exist
        if frame_path not in final_train_set_by_frame_cp.keys():
            # print(frame_path)
            final_train_set_by_frame_cp[frame_path] = {}
        # Match bbox from gt_track3 to list of bbox in track5    
        matchee = match_bbox(bbox_cor, final_train_set_by_frame_cp[frame_path].keys(), matching_threhold)
        
        if matchee == bbox_cor: #Can't match
            not_match_counter +=1
            final_train_set_by_frame_cp[frame_path][bbox_cor] = ['z.unknow']*9
            final_train_set_by_frame_cp[frame_path][bbox_cor][1] = vehId
        else:
            value = copy.deepcopy(final_train_set_by_frame_cp[frame_path][matchee])
            value [1] = vehId
                        
            # Update final_train_set_by_frame_cp
            final_train_set_by_frame_cp[frame_path][bbox_cor] = value
            final_train_set_by_frame_cp[frame_path][bbox_cor][0] = matchee
            del(final_train_set_by_frame_cp[frame_path][matchee])
            
            # add values to tracks
            value[0] = ()
            tracks[int(vehId)].append(value)

        
not_match_counter/total_count

0.2047763385351299

In [31]:
total_count

253374

In [32]:
not_match_counter

51885

In [33]:
total_count-not_match_counter

201489

In [34]:
# Update tracks by the max frequency value of bboxes belonging to the same VehID (track3)
max_tracks = {}
for k,track in tracks.items():
    # print (track)
    if len(track) == 0: 
        # max_tracks[k]=[]
        continue
    max_fre_val = max(track,key=track.count)
    max_tracks[k] = max_fre_val
    
max_tracks[5]

[(),
 5,
 'van.suv',
 False,
 'black',
 True,
 'A black van turns right after a red vehicle keeps straight in an intersection.',
 'A black SUV turns right.',
 'A big car turning right.']

In [35]:
# Update value for not iou matching bbox by the value of bbox belonging to the same trackfor 
max_fre_matching_counter = 0
for frame_path, bboxes in final_train_set_by_frame_cp.items():
    for bbox, value in bboxes.items():
        if value[-1] == 'z.unknow':
            if final_train_set_by_frame_cp[frame_path][bbox][1] in max_tracks.keys():
                max_fre_matching_counter +=1
                final_train_set_by_frame_cp[frame_path][bbox] = max_tracks[final_train_set_by_frame_cp[frame_path][bbox][1]] 
            
max_fre_matching_counter

20981

In [36]:
not_match_counter = not_match_counter - max_fre_matching_counter
not_match_counter

30904

In [37]:
(1-0.12196989430644029)

0.8780301056935598

In [38]:
# Percentage of Unlabel bboxes
not_match_counter/total_count

0.12196989430644029

In [39]:
order_final_train_set_by_frame_cp = {key:final_train_set_by_frame_cp[key] for key in sorted(final_train_set_by_frame_cp.keys())}
# order_final_train_set_by_frame_cp

### Check if data is enough ?
 => missing 20% of track3 bboxes

### Extract frame image with annotation from video

In [40]:
%pwd

'/mnt/d/coding/LMMCT/data/AIC21_Track3_MTMC_Tracking'

In [41]:
# Convert bbox key to string
count = 0
order_final_train_set_by_frame_cp1 = copy.deepcopy(order_final_train_set_by_frame_cp)
for frame_path, bboxes in order_final_train_set_by_frame_cp.items():
    # print (bboxes)
    for bbox, att in bboxes.items():
        count+=1
        # print (bbox)
        bbox_str = [str(val) for val in bbox]
        # print (bbox_str)
        bbox_str = '_'.join(bbox_str)
        # print (bbox_str)
        order_final_train_set_by_frame_cp1[frame_path][bbox_str] = copy.deepcopy(att)
        del(order_final_train_set_by_frame_cp1[frame_path][bbox])
        
        # if count == 10: break
        
count

254301

In [42]:
# Export data to json
json_path = './mapper_nls2Track3.json'
with open(json_path, 'w') as f :
    json.dump(order_final_train_set_by_frame_cp1,f)

In [43]:
# # Double check json load
# import pickle
# with open (json_path, 'r') as f:
#     tmp = json.load(f)
# tmp

In [44]:
COLORS_10 =[(144,238,144),(178, 34, 34),(221,160,221),(  0,255,  0),(  0,128,  0),(210,105, 30),(220, 20, 60),
            (192,192,192),(255,228,196),( 50,205, 50),(139,  0,139),(100,149,237),(138, 43,226),(238,130,238),
            (255,  0,255),(  0,100,  0),(127,255,  0),(255,  0,255),(  0,  0,205),(255,140,  0),(255,239,213),
            (199, 21,133),(124,252,  0),(147,112,219),(106, 90,205),(176,196,222),( 65,105,225),(173,255, 47),
            (255, 20,147),(219,112,147),(186, 85,211),(199, 21,133),(148,  0,211),(255, 99, 71),(144,238,144),
            (255,255,  0),(230,230,250),(  0,  0,255),(128,128,  0),(189,183,107),(255,255,224),(128,128,128),
            (105,105,105),( 64,224,208),(205,133, 63),(  0,128,128),( 72,209,204),(139, 69, 19),(255,245,238),
            (250,240,230),(152,251,152),(  0,255,255),(135,206,235),(  0,191,255),(176,224,230),(  0,250,154),
            (245,255,250),(240,230,140),(245,222,179),(0,139,139),(143,188,143),(255,  0,  0),(240,128,128),
            (102,205,170),( 60,179,113),( 46,139, 87),(165, 42, 42),(178, 34, 34),(175,238,238),(255,248,220),
            (218,165, 32),(255,250,240),(253,245,230),(244,164, 96),(210,105, 30)]
len(COLORS_10)

75

### Export csv file for double check 

In [45]:
import csv
import pandas as pd
scenario = ['S01', 'S02','S03','S04','S05']
cams = range(41)
header = ['frame_path', 'bbox', 'VehId','Type', 'Type_confidence', 'Color', 'Col_confidence', 'Script1', 'Script2', 'Script3' ]
checker_dir = 'checker_nls'
Path(checker_dir).mkdir(parents=True, exist_ok=True)
row_counter = 0
for cam in cams:
    print(f'cam{cam}')
    rows = []
    path = Path(checker_dir) / f'checker_c{cam:02d}.xlsx'
    for frame_path, bboxes in order_fin  al_train_set_by_frame_cp1.items():
        if int(frame_path.split('/')[2][1:]) != cam:
            continue
        for bbox, value in bboxes.items():
            row = []
            row.append(frame_path)
            row.append(bbox),
            for i in range(1,len(value)):
                row.append(value[i])   
            rows.append(row)
    if len(rows) != 0:
        row_counter+=len(rows)
        df = pd.DataFrame(rows, columns = header)
        df.to_excel(str(path))
row_counter

cam0
cam1
cam2
cam3
cam4
cam5
cam6
cam7
cam8
cam9
cam10
cam11
cam12
cam13
cam14
cam15
cam16
cam17
cam18
cam19
cam20
cam21
cam22
cam23
cam24
cam25
cam26
cam27
cam28
cam29
cam30
cam31
cam32
cam33
cam34
cam35
cam36
cam37
cam38
cam39
cam40


254301

In [46]:
order_final_train_set_by_frame_cp1

{'train/S01/c001/original_img/000055.jpg': {'1581_346_339_158': [(),
   34,
   'pickup',
   True,
   'blue',
   True,
   'A blue pickup truck keeps straight at an intersection.',
   'A blue pickup drives through the intersection.',
   'A blue Pickup Truck running down the street.']},
 'train/S01/c001/original_img/000056.jpg': {'1509_347_411_160': [(),
   34,
   'pickup',
   True,
   'blue',
   True,
   'A blue pickup truck keeps straight at an intersection.',
   'A blue pickup drives through the intersection.',
   'A blue Pickup Truck running down the street.']},
 'train/S01/c001/original_img/000057.jpg': {'1445_345_432_157': [(),
   34,
   'pickup',
   True,
   'blue',
   True,
   'A blue pickup truck keeps straight at an intersection.',
   'A blue pickup drives through the intersection.',
   'A blue Pickup Truck running down the street.']},
 'train/S01/c001/original_img/000058.jpg': {'1378_343_433_158': [(),
   34,
   'pickup',
   True,
   'blue',
   True,
   'A blue pickup truck kee

In [47]:
output_dir ='new'
img_path ='train/S01/c001/original_img/000055.jpg'
img_path = Path(img_path)
str_path = str(img_path).replace('train', f'train_{output_dir}')
str_path = str_path.replace('validation', f'validation_{output_dir}')
output_path = Path(str_path)
output_path

PosixPath('train_new/S01/c001/original_img/000055.jpg')